In [1]:
!pip install beautifulsoup4 
!pip install selenium
!pip install selenium bs4
#!apt-get update
!apt install chromium-chromedriver
!pip install scrapingbee

     |████████████████████████████████| 968 kB 6.7 MB/s 
     |████████████████████████████████| 138 kB 48.8 MB/s 
     |████████████████████████████████| 359 kB 48.4 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 4.0 MB 42.8 MB/s 
     |████████████████████████████████| 58 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following addi

In [2]:
# -*- coding: utf-8 -*-
"""
@author: rafam
"""
from bs4 import BeautifulSoup
import pandas as pd

import requests




headers = { "accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "user-agent" : "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 OPR/74.0.3911.107"}


data = {
        'prof_name': [],
        'prof_citations': [],
        'prof_url': [],
        }

def export_table_and_print(data, uni_name):
    table = pd.DataFrame(data, columns=['prof_name','prof_citations','prof_url'])
    table.index = table.index + 1

    table.to_csv(f'{uni_name}_professors.csv', index = False)
    
    print('Scraping done... Here are the results')
    
    print(table)



def get_university_profile(profile):
    
    prof_name = profile.find('h3', class_ = 'gs_ai_name')
    
    prof_citations = profile.find('div', class_ = "gs_ai_cby")
    
    prof_url = profile.find('h3').a['href']

    data['prof_name'].append(prof_name.text)
    data['prof_citations'].append(prof_citations.text[9:])
    data['prof_url'].append(prof_url)



def parse_page(next_url):
    response = requests.get(url, headers=headers)
    
    if response.status_code == requests.codes.ok:
        soup = BeautifulSoup(response.content, 'lxml')
    
        profiles = soup.findAll('div', class_='gs_ai gs_scl gs_ai_chpr')
    
        for profile in profiles:
            get_university_profile(profile)
            

        
        next_author = soup.find("button", {"aria-label": "Next"} )
        next_author = next_author['onclick'][-29:-17]
        #print(next_author)
        return next_author


# ________________________________________

# Get all prof data from university
organization = 16589592566991147599
#11070868180552756631 # FIU
#16589592566991147599 # UM
#9642744976452465328 # Purdue 
#5315230152465104173 # UF
#18168328457749716238 #Upenn
#14823011757688503605 #U Texas Austin 
#14719441169074583706 # Pitt
#15204255840385501358 # Georgia Tech
#8539678734835078480 #Stanford
#14108176128635076915 #UCLA
#17070613643812647106 #FSU



org = [11070868180552756631] #, 16589592566991147599, 9642744976452465328, 5315230152465104173, 18168328457749716238, 14823011757688503605, 14719441169074583706, 15204255840385501358, 8539678734835078480, 14108176128635076915, 17070613643812647106]


url = 'https://scholar.google.com/citations?view_op=view_org&org=' + str(organization) + '&hl=en&oi=io'

next_page = parse_page(url)

# Getting University Name
response = requests.get(url, headers=headers)

if response.status_code == requests.codes.ok:
    soup = BeautifulSoup(response.content, 'lxml')

    uni = soup.find('h2', class_='gsc_authors_header').text[:-11].replace(' ','_')
    print(uni)


for i in org:
  #Change to 29 for 300 professors
  counter = 29
  for i in range(counter):    
      
      url = f'https://scholar.google.com/citations?view_op=view_org&hl=en&org=' + str(i) + '&after_author=' + next_page + '&astart=10'
      #print('The next page url is: \n', next_page, '\n')
      next_page = parse_page(url)
      
  export_table_and_print(data, uni)




University_of_Miami


TypeError: ignored

In [ ]:
from pandas import *
from pathlib import Path 
from selenium import webdriver
import numpy as np
import time
import random
import re
import requests

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException

base_url = 'https://scholar.google.com'
prof_ur = '/citations?hl=en&user=Gn5Fc8kAAAAJ'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled') 


paper_data = {
        'name': [],
        'authors': [],
        'citations': [],
        'webpage': []
        }

def export_table_and_print_paper(data, soup):

  prof_uni = soup.find('a', class_ = 'gsc_prf_ila')
  try:
    prof_uni = prof_uni.text.replace(' ', '_').replace(',','')
  except:
    print("error")

  try: 
    prof_name = soup.find('div', id = 'gsc_prf_in')
    prof_name = prof_name.text.replace(' ', '_').replace('.','').replace(',','')
    print(prof_name)
  except:
    print("error")


  table = pd.DataFrame(data, columns=['name','authors','citations','webpage'])
  table.index = table.index + 1

  filepath = Path(f'Papers/{prof_uni}/{prof_name}.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)
  table.to_csv(filepath, sep = ',', index = False)



  print('Scraping done... Here are the results')
  
  print(table)


def get_paper_info(paper):
  name = paper.find('a', class_ = 'gsc_a_at').text.replace(',','')

  p_webpage = base_url + paper.find('td', class_ = 'gsc_a_t').a['href']

  authors = paper.find('div', class_= 'gs_gray').text
  if authors[-3:] == '...':
    paper_page = requests.get(p_webpage, headers=headers)

    if paper_page.status_code == requests.codes.ok:
      soup1 = BeautifulSoup(paper_page.content, 'lxml')

      authors = soup1.find('div', class_='gsc_oci_value').text

  citations = paper.find('td', class_ = 'gsc_a_c').text
  if bool(citations) == False:
    citations = '0'
  citations = re.sub("[^0-9]", "", citations)
  if citations[-1] == '*':
    citations = citations[:-1]

  paper_data['name'].append(name)
  paper_data['authors'].append(authors)
  paper_data['citations'].append(citations)
  paper_data['webpage'].append(p_webpage)



def get_url(url):
  url = base_url + url

  wd = webdriver.Chrome('chromedriver',options=options) #
  wd.get(url)

  for i in range(6):
    try:
      wd.find_element(By.ID, "gsc_bpf").click()
      time.sleep(random.uniform(4,8))
    except NoSuchElementException:
      print("Exception Handled")

  response = requests.get(url, headers=headers)

  if response.status_code == requests.codes.ok:
    print('still good')        
    soup = BeautifulSoup(response.content, 'lxml')

    papers = soup.findAll('tr', class_='gsc_a_tr')

    for paper in papers:
      get_paper_info(paper)

    export_table_and_print_paper(paper_data, soup)

  #wd.close()


data = read_csv('University_of_Miami_professors.csv')

url_list = data['prof_url'].tolist()

# url_list = ['/citations?hl=en&user=onj9e_wAAAAJ']
print(url_list)

counter = 0
for url in url_list:
  print(counter, ' :', url)
  counter+=1
  get_url(url)
  paper_data = {
        'name': [],
        'authors': [],
        'citations': [],
        'webpage': [],
        }


In [18]:
from pandas import *
from pathlib import Path 
from selenium import webdriver
import numpy as np
import time
import random
import re

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException

base_url = 'https://scholar.google.com'
prof_ur = '/citations?hl=en&user=Gn5Fc8kAAAAJ'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled') 


paper_data = {
        'name': [],
        'authors': [],
        'citations': [],
        'webpage': []
        }

def export_table_and_print_paper(data, soup):

  prof_uni = soup.find('a', class_ = 'gsc_prf_ila')
  try:
    prof_uni = prof_uni.text.replace(' ', '_').replace(',','')
  except:
    print(soup)

  try: 
    prof_name = soup.find('div', id = 'gsc_prf_in')
    prof_name = prof_name.text.replace(' ', '_').replace('.','').replace(',','')
    print(prof_name)
  except:
    print(soup)


  table = pd.DataFrame(data, columns=['name','authors','citations','webpage'])
  table.index = table.index + 1

  filepath = Path(f'{prof_uni}/{prof_name}.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)
  table.to_csv(filepath, sep = ';', index = False)



  print('Scraping done... Here are the results')
  
  print(table)


def get_paper_info(paper):
  name = paper.find('a', class_ = 'gsc_a_at').text.replace(',','')

  p_webpage = base_url + paper.find('td', class_ = 'gsc_a_t').a['href']

  authors = paper.find('div', class_= 'gs_gray').text
  if authors[-3:] == '...':
    paper_page = requests.get(p_webpage, headers=headers)

    if paper_page.status_code == requests.codes.ok:
      soup1 = BeautifulSoup(paper_page.content, 'lxml')

      authors = soup1.find('div', class_='gsc_oci_value').text

  citations = paper.find('td', class_ = 'gsc_a_c').text
  if bool(citations) == False:
    citations = '0'
  citations = re.sub("[^0-9]", "", citations)
  if citations[-1] == '*':
    citations = citations[:-1]

  paper_data['name'].append(name)
  paper_data['authors'].append(authors)
  paper_data['citations'].append(citations)
  paper_data['webpage'].append(p_webpage)



def get_url(url):
  url = base_url + url

  wd = webdriver.Chrome('chromedriver',options=options) #
  wd.get(url)

  for i in range(6):
    try:
      wd.find_element(By.ID, "gsc_bpf").click()
      time.sleep(random.uniform(4,8))
    except NoSuchElementException:
      print("Exception Handled")

  if response.status_code == requests.codes.ok:
          
    soup = BeautifulSoup(wd.page_source, 'lxml')

    papers = soup.findAll('tr', class_='gsc_a_tr')

    for paper in papers:
      get_paper_info(paper)

    export_table_and_print_paper(paper_data, soup)

  wd.close()


data = read_csv('University_of_Florida_professors.csv')

url_list = data['prof_url'].tolist()

# url_list = ['/citations?hl=en&user=onj9e_wAAAAJ']
# print(url_list)

for url in url_list:
  print(url)
  get_url(url)
  paper_data = {
        'name': [],
        'authors': [],
        'citations': [],
        'webpage': [],
        }


Streaming output truncated to the last 5000 lines.

Our systems have detected unusual traffic from your computer network.  This page checks to see if it's really you sending the requests, and not a robot.  <a href="#" onclick="document.getElementById('infoDiv').style.display='block';">Why did this happen?</a><br/><br/>
<div id="infoDiv" style="display:none; background-color:#eee; padding:10px; margin:0 0 15px 0; line-height:1.4em;">
This page appears when Google automatically detects requests coming from your computer network which appear to be in violation of the <a href="//www.google.com/policies/terms/">Terms of Service</a>. The block will expire shortly after those requests stop.  In the meantime, solving the above CAPTCHA will let you continue to use our services.<br/><br/>This traffic may have been sent by malicious software, a browser plug-in, or a script that sends automated requests.  If you share your network connection, ask your administrator for help — a different computer 

In [21]:
from google.colab import files
!zip -r /content/UF.zip /content/University_of_Florida
files.download(UF.zip)

updating: content/University_of_Florida/ (stored 0%)
updating: content/University_of_Florida/Dimitri_Bourilkov.csv (deflated 87%)
updating: content/University_of_Florida/Thomas_D_Schmittgen.csv (deflated 73%)


NameError: ignored

In [4]:
!pip install pyspark
!pip install -q findspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 45.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=59c16d5bab113a4e2f7748eaa979583da584d9a1e91f47b5a7b1966970dbaa24
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
from pyspark import SparkConf, SparkContext
import os
sc = SparkContext.getOrCreate()


path = "/content/University_of_Miami"
dir_list = os.listdir(path)
#print(dir_list)

for prof in dir_list:
  if prof != '.ipynb_checkpoints':
    prof = prof.replace(',','')

    #Find prof's last name
    l_name = prof[:-4].split('_')[-1]

    file_exists = os.path.exists(f'{path}/{prof}')
    print(f'Scanning... {path}/{prof}')
    if file_exists == True:
      #Part 1: Find co author with most citaitons
      mydata = sc.textFile(f'University_of_Miami/{prof}')
      header = mydata.first()
      mydata1 = mydata.filter(lambda row: row != header)
      authors_citations = mydata1.map(lambda line: line.split(';')[1:]).map(lambda fields: (fields[1].replace(', ',','), fields[0])).flatMapValues(lambda v: v.split(','))

      # Can be combined into 1 to combine the filter and map
      author = authors_citations.filter(lambda x: l_name in x[1])#.collect()  #Prof Last name!
      #author = sc.parallelize(author)
      co_authors = authors_citations.subtract(author).map(lambda x: (x[1].replace(' ',''), int(x[0])))
      #co_authors.take(1)

      # Co-author count
      co_author_sum = co_authors.reduceByKey(lambda v1, v2: v1 + v2)
      co_author_order = co_author_sum.map(lambda x: (x[1], x[0])).sortByKey(False)

      #print(prof[:-4])
      for item in co_author_order.take(1):
        print(item)
        print(f'The best coauthor for {prof[:-4]} is {item[1]} with {item[0]} citations.')



Scanning... /content/University_of_Miami/Wang_Michael_Y.csv
(22868, 'HVanHaevermaet')
The best coauthor for Wang_Michael_Y is HVanHaevermaet with 22868 citations.
